<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Préparation-des-données-mutations-et-de-leurs-positions." data-toc-modified-id="Préparation-des-données-mutations-et-de-leurs-positions.-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Préparation des données mutations et de leurs positions.</a></span></li><li><span><a href="#Filosofi" data-toc-modified-id="Filosofi-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Filosofi</a></span><ul class="toc-item"><li><span><a href="#Présentation-de-la-base-Filosofi" data-toc-modified-id="Présentation-de-la-base-Filosofi-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Présentation de la base Filosofi</a></span></li><li><span><a href="#Analyse-du-fichier-carroyée-de-200m" data-toc-modified-id="Analyse-du-fichier-carroyée-de-200m-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Analyse du fichier carroyée de 200m</a></span></li><li><span><a href="#Récupération-du-revenu-local" data-toc-modified-id="Récupération-du-revenu-local-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Récupération du revenu local</a></span></li></ul></li></ul></div>

# Préparation des données mutations et de leurs positions.

In [95]:
import pandas as pd
import geopandas as gpd

In [26]:
dossier_donnees = "C:/Users/khale/OneDrive/Documents/Ensae/Python pour le data scientist/Projet/Immo/Données/r32/"

In [110]:
mutation_geomlocmut = gpd.read_file(dossier_donnees + "r32_mutation_geomlocmut.shp")
mutation_geomlocmut = mutation_geomlocmut.rename(columns={"IDMUTATION" : "idmutation"})
mutation_geomlocmut_wgs = mutation_geomlocmut.to_crs({'init' :'epsg:4326'}) 

C:\Users\khale\anaconda3\envs\projet_2A\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [65]:
pt_test = mutation_geomlocmut_wgs.iloc[2]["geometry"]

In [96]:
mutation_brute = pd.read_csv(dossier_donnees + "r32_mutation.csv")
mutation_brute.shape

(569592, 61)

On se retreint également aux mutations ayant eu lieu dans le département du Nord.

In [107]:
code_mutation_nord = list(mutation_brute[mutation_brute["coddep"] == 59]["idmutation"])

In [112]:
mutation_geomlocmut_wgs_nord = mutation_geomlocmut_wgs[mutation_geomlocmut_wgs["idmutation"].isin(code_mutation_nord)]

# Filosofi

## Présentation de la base Filosofi

Depuis plusieurs années, l’Insee diffuse des indicateurs sur les revenus déclarés des ménages à tous les niveaux géographiques jusqu’à l’infracommunal (RFL) ainsi que des indicateurs de revenus disponibles au niveau départemental (RDL).

Le dispositif FiLoSoFi a été mis en œuvre afin de disposer d’indicateurs de niveau de vie, d’inégalité et de pauvreté à un niveau local infra-départemental. Il remplace les dispositifs Revenus Fiscaux Localisés (RFL) et Revenus Disponibles Localisés (RDL).

Les données sont disponibles à une échelle carroyée (carreau de 200m / 1km) [ici](https://www.insee.fr/fr/statistiques/4176290?sommaire=4176305).

## Analyse du fichier carroyée de 200m 

In [6]:
chemin_donnees_filo = r"C:\Users\khale\OneDrive\Documents\Ensae\Python pour le data scientist\Projets\dvf_ensae_sbra\khaled\data\filosofi\metropole\Filosofi2015_carreaux_200m_metropole.shp".replace("\\","/")

In [7]:
filosofi_france = gpd.read_file(chemin_donnees_filo)

In [10]:
filosofi_france.shape

(2289077, 38)

In [14]:
filosofi_nord = filosofi_france[filosofi_france["Depcom"].str[:2] == "59"]

Transformation en WGS84. 

In [22]:
filosofi_nord["geometry"] = filosofi_nord["geometry"].to_crs({'init' :'epsg:4326'})

C:\Users\khale\anaconda3\envs\projet_2A\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


## Récupération du revenu local

In [45]:
pt_test.intersects(filosofi_nord.iloc[0]["geometry"])

False

In [69]:
def is_point_in_poly(point, poly):
    return point.intersects(poly)

In [80]:
#get the number of the poly of the point
#Return None if the point isn't in any polygon.
def get_number_poly(point, polys):
    try: 
        res = list(map(lambda x : is_point_in_poly(point,x) , polys)).index(True)
    except ValueError:
        res = None
    return res

Test de la fonction sur 100 points afin d'avoir une idée du timing : 

In [113]:
test_100_pts =mutation_geomlocmut_wgs_nord["geometry"].head(100)

In [117]:
res2 = list(map(lambda x : get_number_poly(x, filosofi_nord["geometry"]), test_100_pts))

In [118]:
res2

[15604,
 35858,
 8912,
 25461,
 9078,
 None,
 38206,
 13308,
 8912,
 16810,
 19943,
 18599,
 12004,
 2293,
 15193,
 21704,
 33369,
 32433,
 43931,
 20229,
 12033,
 21893,
 16583,
 43564,
 45023,
 34961,
 29000,
 19960,
 29880,
 30846,
 42374,
 18409,
 8912,
 32040,
 25268,
 17002,
 42832,
 36425,
 44504,
 39782,
 15757,
 23301,
 44461,
 40390,
 42055,
 21704,
 23633,
 14728,
 37106,
 41351,
 34415,
 53,
 22061,
 26906,
 17747,
 22063,
 28059,
 34417,
 15193,
 34961,
 13666,
 16990,
 44202,
 33523,
 28413,
 41800,
 6181,
 36963,
 37505,
 20465,
 37327,
 33524,
 45142,
 30198,
 21697,
 21699,
 35613,
 26989,
 33369,
 37233,
 33523,
 33280,
 42444,
 39693,
 30660,
 35581,
 45593,
 26196,
 21702,
 37505,
 44461,
 20229,
 24136,
 25254,
 26471,
 44808,
 16802,
 39694,
 37233,
 33523]